# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 


## Import Packages

In [1]:
#importing some useful packages
import matplotlib.image as mpimg
import numpy as np
import cv2
from moviepy.editor import VideoFileClip

%matplotlib inline

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [2]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Build a Lane Finding Pipeline

My pipeline consisted of the following steps: 

* Converting to grayscale and blurring to remove small unwanted artefacts
* Increasing contrast by pushing the highlights 
* Detecting the edges using a canny edge detector
* Masking the unwanted parts of the image
* Detecting lines using a hough line detector
* Filtering and classifying the lines
* Fitting and drawing two lines (one for each side of the lane)

In [3]:
# Define the Hough transform parameters
rho = 2
theta = np.pi/90
threshold = 15
min_line_length = 200
max_line_gap = 150

# Define the canny edge detection parameters
blur_kernel = 13
canny_low_threshold = 50
canny_high_threshold = 150

# Define the detection horizon
y_cut = .62

# The coordinates of the lines
global prev_bottom_left_x
global prev_bottom_right_x
global prev_top_left_x
global prev_top_right_x

prev_bottom_left_x = 0
prev_bottom_right_x = 0
prev_top_left_x = 0
prev_top_right_x = 0

# This function processes each frame from the video individually
def process_image(image):
    global prev_bottom_left_x
    global prev_bottom_right_x
    global prev_top_left_x
    global prev_top_right_x

    imshape = image.shape


    ## Conversion to grayscale and blurring
    gray = gaussian_blur(grayscale(image),blur_kernel)

    ## Increase contrast
    highlights = (gray[:,:] > 180)
    gray[highlights] = 255

    ## Detect edges
    edges = canny(gray,canny_low_threshold,canny_high_threshold)

    ## Mask the unwanted parts of the image
    vertices = np.array([[ (0, imshape[0]),
                           (imshape[1] * .45, imshape[0] * y_cut),
                           (imshape[1] * .55, imshape[0] * y_cut),
                           (imshape[1], imshape[0]) ]], dtype=np.int32)

    masked = region_of_interest(edges, vertices)

    ## Detect lines
    lines = cv2.HoughLinesP(masked, rho, theta, threshold, np.array([]),
                                    min_line_length, max_line_gap)

    line_image = np.copy(image)*0

    left_x = []
    left_y = []
    right_x = []
    right_y = []

    ## Filter and classify lines
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                slope = np.abs((y2 - y1) / (x2 - x1))
                if(slope > .5):
                    if (x1 < x2 and y1 > y2) or (x2 < x1 and y2 > y1) :
                        # left line
                        left_x.append(x1)
                        left_x.append(x2)
                        left_y.append(y1)
                        left_y.append(y2)
                    else:
                        # right line
                        right_x.append(x1)
                        right_x.append(x2)
                        right_y.append(y1)
                        right_y.append(y2)

    ## Fit and draw the two lines

    # Left line
    top_left_y = int(imshape[0] * y_cut)
    if(len(left_x) and len(left_y)):
        left_slope, left_intersect = np.polyfit(left_x, left_y, 1)
        bottom_left_x = int((imshape[0] - left_intersect) / left_slope)
        top_left_x = int((top_left_y - left_intersect) / left_slope)
        prev_bottom_left_x = bottom_left_x
        prev_top_left_x = top_left_x
    else:
        bottom_left_x = prev_bottom_left_x
        top_left_x = prev_top_left_x

    # Right line
    top_right_y = int(imshape[0] * y_cut)
    if (len(right_x) and len(right_y)):
        right_slope, right_intersect = np.poly1d(np.polyfit(right_x, right_y, 1))
        bottom_right_x = int((imshape[0] - right_intersect) / right_slope)
        top_right_x = int((top_right_y - right_intersect) / right_slope)
        prev_bottom_right_x = bottom_right_x
        prev_top_right_x = top_right_x
    else:
        bottom_right_x = prev_bottom_right_x
        top_right_x = prev_top_right_x

    # Draw the lines
    draw_lines(line_image, [[[bottom_left_x, imshape[0], top_left_x, top_left_y],
                            [bottom_right_x, imshape[0], top_right_x, top_right_y]]], thickness=10)



    # Combine the lines with the original video
    full_image = cv2.addWeighted(image, 0.7 , line_image, 1, 0)
    return full_image

## Test on Videos

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

Then on a third video:

`challenge.mp4`

In [4]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

Let's try the one with the solid white lane on the right first ...

In [5]:
white_output = 'white-output.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white-output.mp4
[MoviePy] Writing video white-output.mp4


100%|█████████▉| 221/222 [00:05<00:00, 39.64it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white-output.mp4 

CPU times: user 2.89 s, sys: 419 ms, total: 3.31 s
Wall time: 6.34 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [6]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [7]:
yellow_output = 'yellow-output.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow-output.mp4
[MoviePy] Writing video yellow-output.mp4


100%|█████████▉| 681/682 [00:17<00:00, 40.05it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow-output.mp4 

CPU times: user 9.05 s, sys: 1.32 s, total: 10.4 s
Wall time: 17.7 s


In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [9]:
challenge_output = 'challenge-output.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video challenge-output.mp4
[MoviePy] Writing video challenge-output.mp4


100%|██████████| 251/251 [00:12<00:00, 20.20it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge-output.mp4 

CPU times: user 5.98 s, sys: 1.8 s, total: 7.78 s
Wall time: 14.1 s


In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))